In [9]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision/research'

In [10]:
%cd ../

/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision


In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_url: str
    local_data_file: Path
    unzip_dir: Path

In [40]:
from cnnClassifier.constants import *
import os
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir, 
            dataset_url=config.dataset_url,
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
            
        )
        return data_ingestion_config

In [41]:
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
from zipfile import ZipFile
import wget

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            # filename = wget.download(self.config.dataste_url, out=self.config.local_data_file)
            filename, headers = request.urlretrieve(
                url = self.config.dataset_url,
                filename = self.config.local_data_file
            )    
            logger.info(f"{filename} download!")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        with ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [43]:
STAGE_NAME = "Data Ingestion stage"

try:
    logger.info(f">>>>>>>> stage {STAGE_NAME} started <<<<<<<<<<<<")
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_dataset()
    data_ingestion.extract_zip_file()
    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<<<<\n\nx===============x")

except Exception as e:
    logger.exception(e)
    raise e


[2023-10-09 12:36:43,486: INFO: 539176610: >>>>>>>> stage Data Ingestion stage started <<<<<<<<<<<<]
[2023-10-09 12:36:43,490: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-09 12:36:43,492: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-09 12:36:43,493: INFO: common: created directory at: artifacts]
[2023-10-09 12:36:43,495: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-09 12:38:29,019: INFO: 4252441191: artifacts/data_ingestion/data.zip download!]
[2023-10-09 12:38:30,407: INFO: 539176610: >>>>>> stage Data Ingestion stage completed <<<<<<<<<

x===============x]
